In [9]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
import urllib.request
from datetime import datetime
from tqdm import tqdm
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed


In [10]:
hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [3]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "lxml")

    div = soup.find_all('div', attrs = {"class":"read-title"})
    
    if len(div) == 0:
        return

    for link in soup.find_all('div', attrs = {"class":"read-title"}):

        href = link.find('a').get('href')

        hrefs.append(href)
    

In [ ]:
max_worker = 10


for t in range(1):
    
    hrefs = []
    url = 'https://www.utusansarawak.com.my'
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'paged' in a_]
    max_page = max([int(a_.split('?paged=')[1]) for a_ in pgno])
    print(max_page)

    
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}?paged={a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()
    
    hrefs = list(set(hrefs))
    with open(f'utusansarawak-link.json', 'a') as f:
        json.dump(hrefs, f)

In [23]:
hrefs = list(set(hrefs))

In [24]:
len(hrefs)

13327

In [10]:
with open('utusansarawak-link.json', 'a') as f:
    json.dump(hrefs, f)

In [22]:
with open('utusansarawak-link.json') as fopen:
    hrefs = json.load(fopen)

In [16]:
def process_url(x):
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            
            if r.status_code == 508:
                time.sleep(20)
                continue
                
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('h1', class_="entry-title").text
        h = soup.find('div', class_="entry-content read-details")
        p = [p.text for p in h.find_all("p")]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x, 'headline': headline, 'content': p}
    
    return data

In [19]:
max_workers = 3

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in not_in_list2]
    
    for future in tqdm(futures, total=len(not_in_list2)):
        result = future.result()
        if result:
            with open('utusansarawak.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

  2%|███▋                                                                                                                                                                             | 13/619 [00:21<17:03,  1.69s/it]

error in link:https://www.utusansarawak.com.my/?tag=dr-awang-azman
'NoneType' object has no attribute 'text'


 25%|███████████████████████████████████████████▌                                                                                                                                    | 153/619 [04:10<20:22,  2.62s/it]

error in link:https://www.utusansarawak.com.my/?tag=dr-suffian-mansor
'NoneType' object has no attribute 'text'


 26%|█████████████████████████████████████████████▏                                                                                                                                  | 159/619 [04:20<13:20,  1.74s/it]

...

 33%|█████████████████████████████████████████████████████████▍                                                                                                                      | 202/619 [05:23<14:14,  2.05s/it]

error in link:https://www.utusansarawak.com.my/?tag=adi-badiozaman-tuah
'NoneType' object has no attribute 'text'


 42%|█████████████████████████████████████████████████████████████████████████▉                                                                                                      | 260/619 [07:05<08:22,  1.40s/it]

error in link:https://www.utusansarawak.com.my/?page_id=38700
'NoneType' object has no attribute 'find_all'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 619/619 [16:15<00:00,  1.58s/it]


In [20]:
import jsonlines

# Open the JSONL file and extract URLs into a list
file_path = 'utusansarawak.jsonl'
url_list1 = []
with jsonlines.open(file_path) as reader:
    for obj in reader:
        url_list1.append(obj['url'])



# Find URLs in list 1 that are not in list 2
not_in_list2 = [url for url in hrefs if url not in url_list1]

In [21]:
len(not_in_list2)

5